In [ ]:
import json,csv,re,sys
import unicodedata
# convert to xml
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString

In [ ]:
# load wordlemma_grc.json: word - lemma correspondences generated from files in pta_data 
data_file = open('/home/stockhausen/Dokumente/projekte/pta_lexika/wordlemma_grc.json')    
lemma_data = json.load(data_file)
# load LSJ and save to json
with open('/home/stockhausen/Dokumente/projekte/pta_lexika/sources/LSJ-6.5.1.txt', encoding='utf-8-sig') as file:
    fnames = ['w', 'd']
    reader = csv.DictReader(file, delimiter="\t", fieldnames=fnames)
    lsj = []
    for line in reader:
        lsj.append(line)
with open('/home/stockhausen/Dokumente/projekte/pta_lexika/LSJ.json', 'w') as fout:
    json.dump(lsj, fout, indent=4, ensure_ascii=False)  
# load Pape and save to json
csv.field_size_limit(500000)
with open('/home/stockhausen/Dokumente/projekte/pta_lexika/sources/Pape-4.6b.txt', encoding='utf-8-sig') as file:
    fnames = ['w', 'd']
    reader = csv.DictReader(file, delimiter="\t", fieldnames=fnames)
    pape = []
    for line in reader:
        pape.append(line)
with open('/home/stockhausen/Dokumente/projekte/pta_lexika/Pape.json', 'w') as fout:
    json.dump(pape, fout, indent=4, ensure_ascii=False)  
# load TBESG - Translators Brief lexicon of Extended Strongs for Greek - STEPBible.org CC BY and save to json
with open('/home/stockhausen/Dokumente/projekte/pta_lexika/sources/TBESG - Translators Brief lexicon of Extended Strongs for Greek - STEPBible.org CC BY.txt', encoding='utf-8-sig') as file:
    fnames = ['s','t','w','g','m','d']
    reader = csv.DictReader(file, delimiter="\t", fieldnames=fnames)
    TBESG = []
    # remove columns not needed for our purposes
    for line in reader:
        line.pop('s')
        line.pop('t')
        line.pop('g')
        line.pop('m')
        TBESG.append(line)
with open('/home/stockhausen/Dokumente/projekte/pta_lexika/TBESG.json', 'w') as fout:
    json.dump(TBESG, fout, indent=4, ensure_ascii=False)  
# load Bailly 2020 and save to json
with open('/home/stockhausen/Dokumente/projekte/pta_lexika/sources/Bailly2020.3a.txt', encoding='utf-8-sig') as file:
    fnames = ['w', 'd']
    reader = csv.DictReader(file, delimiter="\t", fieldnames=fnames)
    Bailly = []
    for line in reader:
        Bailly.append(line)
    Bailly = Bailly[10:] # remove header
with open('/home/stockhausen/Dokumente/projekte/pta_lexika/Bailly2020.3a.json', 'w') as fout:
    json.dump(Bailly, fout, indent=4, ensure_ascii=False)    

In [ ]:
# convert lexika to xml and write to files
count = 0
for lexikon in [lsj,pape,TBESG,Bailly]:
    count = count+1
    xml = dicttoxml(lexikon, attr_type=False)
    xml2 = xml.decode("utf-8")
    s = re.sub(r'&gt;', r'>', xml2)
    s = re.sub(r'&lt;', r'<', s)
    s = re.sub(r'&quot;', r'"', s)
    s = re.sub(r'&apos;',r'"',s)
    filename = "/home/stockhausen/Dokumente/projekte/pta_lexika/"+str(count)+".xml"
    with open(filename, 'w', encoding='utf-8-sig') as file_open:
        file_open.write(s)

In [ ]:
# Lemma-Liste aus wordlemma.json
lemmas = []
lemmas2 = []
for entry in lemma_data:
    lem = entry["Lemma"]
    lemmas.append(lem)
for i in lemmas:
    s = re.sub(r'[0-9]+', r'', str(i)) # remove numbers from lemma entries
    s = s.split("|") # split entries at |
    lemmas2.append(s)
lemma_list = [item for sublist in lemmas2 for item in sublist]
lemma_list = sorted(list(set(lemma_list)))

In [ ]:
print(len(lemma_list))

In [ ]:
def loadingBar(count,total,size):
    percent = float(count)/float(total)*100
    sys.stdout.write("\r" +str(int(count)).rjust(3,'0')+"/"+str(int(total)).rjust(3,'0') + ' [' + '='*int(percent/10)*size + ' '*(10-int(percent/10))*size + ']')

In [ ]:
lexicon = []
count = 0
for entry in lemma_list:
    count = count+1
    loadingBar(count,len(lemma_list),2)
    l_entry = {}
    lookup = unicodedata.normalize("NFC", entry)
    l_entry["lemma"] = lookup
    l_entry["grc_eng"] = [element["d"] for element in lsj if unicodedata.normalize("NFC", element['w']).lower() == lookup.lower()]
    l_entry["grc_eng2"] = [element["Meaning"] for element in TBESG if unicodedata.normalize("NFC", element['Greek']).lower() == lookup.lower()]
    l_entry["grc_deu"] = [element["d"] for element in pape if unicodedata.normalize("NFC", element['w']).lower() == lookup.lower()]
    l_entry["grc_fra"] = [element["d"] for element in Bailly if unicodedata.normalize("NFC", element['w']).lower() == lookup.lower()]
    lexicon.append(l_entry)
liste = [i for n, i in enumerate(lexicon) if i not in lexicon[n + 1:]]

In [ ]:
with open('/home/stockhausen/Dokumente/projekte/pta_lexika/pta_lexicon_grc.json', 'w') as fout:
    json.dump(lexicon, fout, indent=4, ensure_ascii=False)

### convert json to xml

In [ ]:
data_file = open('/home/stockhausen/Dokumente/projekte/pta_lexika/wordlemma_grc.json')    
lemma_data = json.load(data_file)
# split lemma and morphology to subentries
lemma_data2 = []
for entry in lemma_data:
    lemmata = {}
    lemmata["Word"] = entry["Word"]
    lemmata["Lemma"] = entry["Lemma"].split("|")    
    lemmata["Morphology"] = entry["Morphology"].split("|")
    lemma_data2.append(lemmata)
# convert json2 xml
xml = dicttoxml(lemma_data2, attr_type=False)
dom = parseString(xml)
with open("/home/stockhausen/Dokumente/projekte/pta_lexika/wordlemma_grc.xml", 'w') as file_open:
    file_open.write(dom.toprettyxml())

In [ ]:
data_file2 = open('/home/stockhausen/Dokumente/projekte/pta_lexika/pta_lexicon_grc.json')    
lexicon_data = json.load(data_file2)
xml2 = dicttoxml(lexicon_data, attr_type=False)
xml2 = xml2.decode("utf-8")
s = re.sub(r'&gt;', r'>', xml2)
s = re.sub(r'&lt;', r'<', s)
s = re.sub(r'&quot;', r'"', s)
with open("/home/stockhausen/Dokumente/projekte/pta_lexika/pta_lexicon_grc.xml", 'w', encoding='utf-8-sig') as file_open:
    file_open.write(s)
# anschließend: xmllint --format pta_lexicon_grc.xml -o pta_lexicon_grc.xml